In [ ]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

In [ ]:
conda install -c conda-forge ffmpeg

In [ ]:
from openai import OpenAI

client = OpenAI()

## 1. Text To Speech(audio.speech.create)

- TTS는 컴퓨터 프로그램이나 기기가 텍스트를 인간의 음성처럼 들리는 오디오로 변환하는 과정입니다.
- 이 기술은 음성 합성을 통해 텍스트 데이터를 자연스러운 음성으로 바꿉니다.
- 사용 예시: 오디오북, 음성 안내 시스템, 음성 기반 가상 어시스턴트 등.

**[참고]**

- 공식문서: https://platform.openai.com/docs/guides/text-to-speech


**주요 파라미터**

- `model`: 사용 가능한 TTS 모델 중 하나를 지정합니다. `tts-1` 또는 `tts-1-hd`.
  - 최신 지원모델 확인: https://platform.openai.com/docs/models/tts
- `input`: 오디오를 생성할 텍스트입니다. <span style="color: red;">**최대 길이는 4096자입니다**</span>
- `voice`: 오디오를 생성할 때 사용할 음성입니다. 지원되는 음성은 `alloy`, `echo`, `fable`, `onyx`, `nova`, and `shimmer` 입니다. 음성의 미리듣기는 [여기](https://platform.openai.com/docs/guides/text-to-speech/voice-options) 에서 확인할 수 있습니다.
- `response_format`: 오디오를 입력할 형식입니다. 지원되는 형식은 `mp3`, `opus`, `aac` 및 `flac` 입니다.
- `speed`: 생성된 오디오의 속도입니다. `0.25` 에서 `4.0` 사이의 값을 선택합니다. 기본값은 `1.0` 입니다.


In [ ]:
speech_file_path = "tts_audio.mp3"
Ignore DeprecationWarning (response.stream_to_file)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
input_text = """
 나는 오늘, 내 인생의 새로운 장을 열어갈 것이다. 어제의 나를 뛰어넘어, 더 강하고 현명한 나로 성장할 것이다. 
 모든 순간이 배움의 기회임을 명심하며, 긍정적인 에너지로 하루를 채워갈 것이다. 
 내 꿈을 향해 한 걸음 더 나아가는 오늘, 어떤 어려움이 와도 극복해 나갈 것이다. 
 내 삶의 주인공은 바로 나다. 
"""
response = client.audio.speech.create(
    model="tts-1",
    input=input_text,
    voice="nova",       # voice= "alloy", "echo", "fable", "onyx", "nova", "shimmer" 
    response_format="mp3",
    speed=1.0,
)
response.stream_to_file(speech_file_path)

저장한 오디오 파일을 재생합니다.


In [ ]:
from IPython.display import Audio

Audio(speech_file_path)

## 2. Long text to speech

- 4096자보다 긴 텍스트는 당연히 잘라서 음성으로 변환하고, 각각을 하나의 음성파일로 모아야 합니다.
- 다음은 input text file data\CEO처럼 생각하기.txt을 읽어서 output_speech.mp3 음성파일로 변환하는 것입니다. 
- 이를 위해서는 ffmpeg를설치하고, 경로를 지정해주어야 합니다. 

### ffmpeg 역할
- ffmpeg.7z 파일이 data 디렉토리에 있습니다.
- Change the  format of a video/audio file
- Extract audio from video file
- Merge audio and video streams
- Change the bitrate of a video/audio file
- Create GIF from a video file
- Extract still images from a video file
- To embed subtitles into a video file
- To compress or resize a video/audio file
- Record a live stream

In [ ]:
import os
import time
from openai import OpenAI
from pydub import AudioSegment
from IPython.display import Audio, display

# Ignore DeprecationWarning (response.stream_to_file)
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


def split_text_for_tts(text, max_length=4000):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        if len(' '.join(current_chunk)) + len(word) + 1 <= max_length:
            current_chunk.append(word)
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

def safe_remove(file_path, max_attempts=5, delay=1):
    for attempt in range(max_attempts):
        try:
            os.remove(file_path)
            return
        except PermissionError:
            if attempt < max_attempts - 1:
                time.sleep(delay)
            else:
                print(f"Warning: Unable to remove temporary file {file_path}")

def text_file_to_speech(input_file_path, output_file_path, voice="alloy"):
    client = OpenAI()
    
    # 텍스트 파일 읽기
    with open(input_file_path, 'r', encoding='utf-8') as file:
        text = file.read()
    
    # 텍스트를 청크로 분할
    chunks = split_text_for_tts(text)
    total_chunks = len(chunks)
    
    print(f"텍스트가 총 {total_chunks}개의 청크로 나뉘었습니다.")
    
    # 각 청크를 음성으로 변환
    audio_segments = []
    for i, chunk in enumerate(chunks, 1):
        print(f"처리 중: 청크 {i}/{total_chunks}")
        response = client.audio.speech.create(
            model="tts-1",
            voice=voice,
            input=chunk
        )
        
        # 임시 파일로 저장
        temp_file = f"temp_audio_{i}.mp3"
        response.stream_to_file(temp_file)
        
        try:
            # AudioSegment로 로드
            audio_segment = AudioSegment.from_mp3(temp_file)
            audio_segments.append(audio_segment)
        except Exception as e:
            print(f"Error processing chunk {i}: {e}")
        finally:
            # 임시 파일 삭제 시도
            safe_remove(temp_file)
        
        print(f"청크 {i}/{total_chunks} 변환 완료")
    
    if not audio_segments:
        print("오디오 세그먼트가 생성되지 않았습니다. 프로세스를 종료합니다.")
        return
    
    print("모든 청크 변환 완료. 오디오 파일 결합 중...")
    
    # 모든 오디오 세그먼트 결합
    combined_audio = sum(audio_segments)
    
    # 최종 파일로 내보내기
    combined_audio.export(output_file_path, format="mp3")
    
    print(f"변환 완료. 파일 저장됨: {output_file_path}")
    print(f"총 {total_chunks}개의 청크가 처리되었습니다.")
    
    # 생성된 오디오 재생
    display(Audio(output_file_path))

# 실행
input_file = "data/CEO처럼 생각하기.txt"  # 긴 텍스트가 들어있는 파일
output_file = "output_speech.mp3"  # 결과 음성 파일

text_file_to_speech(input_file, output_file)

In [ ]:
from IPython.display import Audio

Audio(output_file)

## 3. Speech To Text(autio.transcriptons.create)

- STT는 사람의 말소리를 텍스트로 변환하는 기술입니다.
- 이는 음성 인식을 통해 구어체 언어를 캡처하고 이를 기록 가능한 형태의 텍스트로 변환합니다. (max 25MB)
- 사용예시: 음성 명령 입력, 자동 회의록 작성, 음성 기반 검색 시스템 등.

**[참고]**

- 공식문서: https://platform.openai.com/docs/guides/speech-to-text
- 파일 업로드는 현재 25MB로 제한되어 있으며, 지원되는 입력 파일 형식은 `mp3`, `MP4`, `MPEG`, `MPGA`, `M4A`, `WAV`, `WEBM` 입니다.


**주요 파라미터**

- `file`: 변환할 오디오 파일 개체(파일 이름이 아님)로, 다음 형식 중 하나입니다: `FLAC`, `MP3`, `MP4`, `MPEG`, `MPGA`, `M4A`, `OGG`, `WAV` 또는 `WEBM`.
- `model`: 현재는 `whisper-1` 모델만 지정 가능합니다.
- `language`: 입력 오디오의 언어입니다. 입력 언어를 ISO-639-1 형식으로 제공하면 정확도와 지연 시간이 개선됩니다.
  - [ISO-639-1 형식](https://en.wikipedia.org/wiki/List_of_ISO_639-1_codes)
- `prompt`: (선택 사항) 모델의 스타일을 안내하거나 이전 오디오 세그먼트를 계속하기 위한 텍스트입니다. 프롬프트는 오디오 언어와 일치해야 합니다.
- `response_format`: 변환된 결과물 출력 형식입니다. 가능한 지정 옵션은 `json`, `text`, `srt`, `verbose_json` 또는 `vtt` 입니다.
- `temperature`: 0에서 1 사이의 샘플링 `temperature` 입니다. 0.8과 같이 값이 높을수록 출력은 더 무작위적이고, 0.2와 같이 값이 낮을수록 출력은 더 집중적이고 결정론적입니다. 0으로 설정하면 모델은 로그 확률을 사용하여 특정 임계값에 도달할 때까지 자동으로 `temperature` 을 높입니다.


In [ ]:
audio_file = open("data/채용면접_샘플_01.wav", "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",
    temperature=0.0,
)

In [ ]:
# 결과물 출력
print(transcript)

## 4. Long Audio file

- 기본적으로 Whisper API는 **25MB 미만의 파일** 만 지원합니다.

- 이보다 긴 오디오 파일이 있는 경우 **25MB 이하의 청크로 나누거나 압축된 오디오 형식을 사용** 해야 합니다.

- 최상의 성능을 얻으려면 문장 중간에 오디오를 분할하면 일부 문맥이 손실될 수 있으므로 분할을 피하는 것이 좋습니다.

- 이를 처리하는 한 가지 방법은 `PyDub` 오픈 소스 Python 패키지를 사용하여 **오디오를 분할** 하는 것입니다.


**샘플 데이터셋(채용면접 인터뷰 데이터)**

- 링크: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=data&dataSetSn=71592


아래의 코드는 오디오 파일을 정해진 시간에 따라 분절하여 별도의 파일로 저장하는 코드입니다.


In [ ]:
!pip install pydub

### 1회차 4시간 강의 내용을 Text로 푸는 것을 연습해보겠습니다. 

In [ ]:
from pydub import AudioSegment
import os

# ffmpeg 경로 설정
AudioSegment.converter = r"E:\\ffmpeg\bin\\ffmpeg.exe"  
AudioSegment.ffprobe = r"E:\\ffmpeg\\bin\\ffprobe.exe"   
# 파일 경로 설정 (절대 경로 사용)
filename = "E:\\course\\04.OpenAI\\data\\20240701-1회차(음성).m4a"
print(filename)
myaudio = AudioSegment.from_file(filename)



In [ ]:
# PyDub 는 밀리초 단위로 시간을 계산합니다.
threehundred_seconds = 5 * 60 * 1000  # (5분 * 60초 * 1000밀리초)
total_milliseconds = myaudio.duration_seconds * 1000  # 전체 길이를 밀리초로 변환
# 전체 길이를 5분으로 나누어서 반복할 횟수를 계산합니다.
total_iterations = int(total_milliseconds // threehundred_seconds + 1)
total_iterations

In [ ]:
# 생성된 파일명을 저장할 리스트
output_filenames = []

for i in range(total_iterations):
    if i < total_iterations - 1:
        # 300초 단위로 오디오를 분할합니다.
        part_of_audio = myaudio[threehundred_seconds * i: threehundred_seconds * (i + 1)]
    else:
        # 마지막은 나머지 전체를 분할합니다.
        part_of_audio = myaudio[threehundred_seconds * i:]

    output_filename = (
        # 예시: 채용면접_샘플_02-(1).mp3, 채용면접_샘플_02-(2).mp3 ...
        f"{filename[:-4]}-({i+1}).mp3"
    )

    # 분할된 오디오를 저장합니다.
    part_of_audio.export(output_filename, format="mp3")
    output_filenames.append(output_filename)

In [ ]:
# 결과물(파일명) 출력
output_filenames

In [ ]:
transcripts = []

for audio_filename in output_filenames:
    audio_file = open(audio_filename, "rb")  # audio file 을 읽어옵니다.

    # transcript 를 생성합니다.
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",  # 모델은 whisper-1 을 사용
        language="ko",  # 한국어를 사용
        response_format="text",  # 결과물은 text 로 출력
        temperature=0.0,
    )

    # 생성된 transcript 를 리스트에 추가합니다.
    transcripts.append(transcript)
    print(audio_filename)

# 전체 transcript 출력(리스트를 문자열로 변환)
final_output = "\n".join(transcripts)

# 파일 경로와 이름을 지정합니다.
file_path = 'output.txt'

# 파일을 쓰기 모드로 열고 내용을 기록합니다.
with open(file_path, 'w', encoding='utf-8') as file:
    file.write(final_output)

print(f"파일이 성공적으로 저장되었습니다: {file_path}")


In [ ]:
import os
from openai import OpenAI
import tiktoken


# GPT-4 토크나이저 초기화
tokenizer = tiktoken.encoding_for_model("gpt-4o")

# 강의를 책 형식으로 변환하는 프롬프트
conversion_prompt = """
당신은 뛰어난 작가입니다. 주어진 강의 내용을 책과 같은 글투로 변환해주세요.
다음 지침을 따라주세요:

1. 강의체의 구어체를 격식 있는 문어체로 바꿔주세요.
2. '여러분'이나 '우리'와 같은 청자 지칭을 제거하고, 객관적인 서술로 바꿔주세요.
3. 강지하되의에서 사용된 예시나 설명은 유, 책에 어울리는 방식으로 재구성해주세요.
4. 중복되는 내용은 정리하고, 논리적 흐름을 강화해주세요.
5. 전문 용어나 중요 개념은 그대로 유지하면서, 필요한 경우 추가 설명을 덧붙여주세요.
6. 강의의 핵심 내용과 구조는 유지하되, 책에 적합한 문단 구성과 전개를 해주세요.
7. 말투는 '~ 했다.", "~ 그것이다." 라고 하지말고, "~ 했습니다.", "~ 그것입니다." 등과 같이 높임말로 바꿔주세요.

최종 결과물은 마치 전문 작가가 쓴 교양 서적처럼 읽히도록 해주세요.
이 내용은 더 큰 텍스트의 일부일 수 있으므로, 자연스럽게 이어지도록 작성해주세요.
"""

def count_tokens(text):
    return len(tokenizer.encode(text))

def chunk_text(text, max_tokens=8000):
    chunks = []
    current_chunk = ""
    current_tokens = 0

    for sentence in text.split(". "):
        sentence_tokens = count_tokens(sentence)
        if current_tokens + sentence_tokens > max_tokens:
            chunks.append(current_chunk.strip())
            current_chunk = sentence
            current_tokens = sentence_tokens
        else:
            current_chunk += sentence + ". "
            current_tokens += sentence_tokens

    if current_chunk:
        chunks.append(current_chunk.strip())

    return chunks

def convert_lecture_to_book(lecture_content):
    response = client.chat.completions.create(
        model="gpt-4o",  
        temperature=0.7,
        messages=[
            {"role": "system", "content": conversion_prompt},
            {"role": "user", "content": lecture_content},
        ]
    )
    return response.choices[0].message.content

# 입력 파일 읽기
input_file = 'output.txt'
with open(input_file, 'r', encoding='utf-8') as file:
    lecture_content = file.read()

# 텍스트를 청크로 나누기
chunks = chunk_text(lecture_content)

print(f"총 {len(chunks)}개의 청크로 나누어 처리합니다.")

# 각 청크 처리
converted_chunks = []
for i, chunk in enumerate(chunks):
    print(f"청크 {i+1}/{len(chunks)} 처리 중...")
    converted_chunk = convert_lecture_to_book(chunk)
    converted_chunks.append(converted_chunk)

# 모든 변환된 청크 합치기
final_book_content = "\n\n".join(converted_chunks)

# 결과를 파일에 저장
output_file = 'book1.txt'
with open(output_file, 'w', encoding='utf-8') as file:
    file.write(final_book_content)

print(f"변환된 내용이 {output_file}에 저장되었습니다.")